Load these Libraries and the data file

In [1]:
!pip install --upgrade category_encoders

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, plot_confusion_matrix, classification_report, f1_score

In [3]:
#Gather data from dataset file
data = pd.read_csv("C:\\Users\\GR\\Documents\\LCITW2022\\T1\\CBD 2214 - Big Data Fundamental Data Storage Networking\\CarEvaluation\\data-set\\car_evaluation.csv",header=None)
data.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [4]:
# Change the column numbers to readable column names
col_names = ['buying','maint','doors','persons','luggage_boot','safety','class']
data.columns = col_names
data.head()

,buying,maint,doors,persons,luggage_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   buying        1728 non-null   object
 1   maint         1728 non-null   object
 2   doors         1728 non-null   object
 3   persons       1728 non-null   object
 4   luggage_boot  1728 non-null   object
 5   safety        1728 non-null   object
 6   class         1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [6]:
def show(data):
  for i in data.columns[1:]:
    print("Feature: {} with {} Levels".format(i,data[i].unique()))

show(data)

Feature: maint with ['vhigh' 'high' 'med' 'low'] Levels
Feature: doors with ['2' '3' '4' '5more'] Levels
Feature: persons with ['2' '4' 'more'] Levels
Feature: luggage_boot with ['small' 'med' 'big'] Levels
Feature: safety with ['low' 'med' 'high'] Levels
Feature: class with ['unacc' 'acc' 'vgood' 'good'] Levels


#### Exploratory Data Analysis using Tableau

In [7]:
data.isnull().sum() 

buying          0
maint           0
doors           0
persons         0
luggage_boot    0
safety          0
class           0
dtype: int64

Tableau Link: https://public.tableau.com/views/EDAonCarEvaluation/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link

In [8]:
%%HTML
<div class='tableauPlaceholder' id='viz1637670871760' style='position: relative'><noscript><a href='#'><img alt='Car Evaluation Exploratory Data Analysis ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ED&#47;EDAonCarEvaluation&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='EDAonCarEvaluation&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ED&#47;EDAonCarEvaluation&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1637670871760');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='1677px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

#### Feature Engineering

In [9]:
data.dtypes

buying          object
maint           object
doors           object
persons         object
luggage_boot    object
safety          object
class           object
dtype: object

In [10]:
encoder = ce.OrdinalEncoder(cols = ['buying','maint','doors','persons','luggage_boot','safety','class'])
data = encoder.fit_transform(data)
data.head()

,buying,maint,doors,persons,luggage_boot,safety,class
0,1,1,1,1,1,1,1
1,1,1,1,1,1,2,1
2,1,1,1,1,1,3,1
3,1,1,1,1,2,1,1
4,1,1,1,1,2,2,1


#### Splitting Data into Train & Test

In [11]:
x = data.drop(['class'], axis = 1)
y = data['class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 45)
print("X_train: {}".format(x_train.shape))
print("X_test: {}".format(x_test.shape))
print("Y_train: {}".format(y_train.shape))
print("Y_test: {}".format(y_test.shape))

X_train: (1209, 6)
X_test: (519, 6)
Y_train: (1209,)
Y_test: (519,)


#### Data Modeling

#### Creating Evaluation Parametric Function

In [12]:
def evaluation_parametrics(y_train,yp_train,y_test,yp_test):
  print("--------------------------------------------------------------------------")
  print("Classification Report for Train Data")
  print(classification_report(y_train, yp_train))
  print("Classification Report for Test Data")
  print(classification_report(y_test, yp_test))
  print("--------------------------------------------------------------------------")
  # Accuracy
  print("Accuracy on Train Data is: {}".format(round(accuracy_score(y_train,yp_train),2)))
  print("Accuracy on Test Data is: {}".format(round(accuracy_score(y_test,yp_test),2)))
  print("--------------------------------------------------------------------------")
  # Precision
  print("Precision on Train Data is: {}".format(round(precision_score(y_train,yp_train,average = "weighted"),2)))
  print("Precision on Test Data is: {}".format(round(precision_score(y_test,yp_test,average = "weighted"),2)))
  print("--------------------------------------------------------------------------")
  # Recall 
  print("Recall on Train Data is: {}".format(round(recall_score(y_train,yp_train,average = "weighted"),2)))
  print("Recall on Test Data is: {}".format(round(recall_score(y_test,yp_test,average = "weighted"),2)))
  print("--------------------------------------------------------------------------")
  # F1 Score
  print("F1 Score on Train Data is: {}".format(round(f1_score(y_train,yp_train,average = "weighted"),2)))
  print("F1 Score on Test Data is: {}".format(round(f1_score(y_test,yp_test,average = "weighted"),2)))
  print("--------------------------------------------------------------------------")


#### 1. Logistics Regression

In [13]:
lr = LogisticRegression(max_iter = 1000,random_state = 50)
lr.fit(x_train,y_train)

yp_train = lr.predict(x_train)
yp_test = lr.predict(x_test)

evaluation_parametrics(y_train,yp_train,y_test,yp_test)

--------------------------------------------------------------------------
Classification Report for Train Data
              precision    recall  f1-score   support

           1       0.88      0.94      0.91       848
           2       0.70      0.60      0.65       267
           3       0.78      0.69      0.73        45
           4       0.54      0.41      0.47        49

    accuracy                           0.84      1209
   macro avg       0.73      0.66      0.69      1209
weighted avg       0.83      0.84      0.83      1209

Classification Report for Test Data
              precision    recall  f1-score   support

           1       0.89      0.92      0.90       362
           2       0.68      0.64      0.66       117
           3       0.75      0.75      0.75        20
           4       0.71      0.50      0.59        20

    accuracy                           0.83       519
   macro avg       0.76      0.70      0.72       519
weighted avg       0.83      0.83    

#### 2. Decision Tree

In [14]:
dt = DecisionTreeClassifier(max_depth = 6,random_state = 50) # Manage max_depth = 6 to ensure operation within best fitting range
dt.fit(x_train,y_train)

yp_train = dt.predict(x_train)
yp_test = dt.predict(x_test)

evaluation_parametrics(y_train,yp_train,y_test,yp_test)

--------------------------------------------------------------------------
Classification Report for Train Data
              precision    recall  f1-score   support

           1       1.00      0.96      0.98       848
           2       0.85      0.88      0.86       267
           3       0.78      0.78      0.78        45
           4       0.61      0.84      0.71        49

    accuracy                           0.93      1209
   macro avg       0.81      0.86      0.83      1209
weighted avg       0.94      0.93      0.93      1209

Classification Report for Test Data
              precision    recall  f1-score   support

           1       0.99      0.96      0.97       362
           2       0.88      0.91      0.89       117
           3       0.89      0.85      0.87        20
           4       0.66      0.95      0.78        20

    accuracy                           0.94       519
   macro avg       0.85      0.92      0.88       519
weighted avg       0.95      0.94    

#### 3. Random Forest

In [15]:
rf = RandomForestClassifier(max_depth = 6,random_state = 50) # Maintain max_depth = 6 equivalent to Dicision Tree range.
rf.fit(x_train,y_train)

yp_train = rf.predict(x_train)
yp_test = rf.predict(x_test)

evaluation_parametrics(y_train,yp_train,y_test,yp_test)

--------------------------------------------------------------------------
Classification Report for Train Data
              precision    recall  f1-score   support

           1       0.99      0.97      0.98       848
           2       0.82      0.96      0.88       267
           3       0.80      0.73      0.77        45
           4       0.95      0.39      0.55        49

    accuracy                           0.94      1209
   macro avg       0.89      0.76      0.79      1209
weighted avg       0.94      0.94      0.93      1209

Classification Report for Test Data
              precision    recall  f1-score   support

           1       0.98      0.96      0.97       362
           2       0.79      0.93      0.85       117
           3       0.93      0.70      0.80        20
           4       0.80      0.40      0.53        20

    accuracy                           0.92       519
   macro avg       0.88      0.75      0.79       519
weighted avg       0.93      0.92    

#### Conclusion
* Implemented Logistic Regression, Decision Tree & Random Forest models.
* Evaluated Evaluation Metrics (Confusion Matrix, Precision, Recall, Accuracy & F1-Score).
* Random Forest performed slightly better out of the 3 models.

Insert table or image of table depicting the best model used (here).